In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import keras
from math import sqrt
import sklearn.metrics
import scipy

In [ ]:
array_of_train_images= np.load('path to the x_train_images_array.npy')
labels_train_array= np.load('path to the y_train_labels_array.npy')
array_of_test_images= np.load('path to the x_test_images_array.npy')
labels_test_array= np.load('path to the y_test_labels_array.npy')

print(array_of_train_images.shape)
print(labels_train_array.shape)
print(array_of_test_images.shape)
print(labels_test_array.shape)

(1748, 250, 250, 3)
(1748,)
(256, 250, 250, 3)
(256,)


In [ ]:
predictions = np.load('path to the predictions of the model.npy')
predictions.shape

(256, 1)

In [ ]:
#BOOSTRAP CONFIDENCE INTERVALS

def bootstrapping_metrics (preds, y_tests, confidence, n_bootstraps, threshold):

    rng_seed= np.random.RandomState(42) 
    
    bootstrapped_sensitivity= []
    bootstrapped_specificity= []
    bootstrapped_precission= []
    bootstrapped_NPV= []  
    bootstrapped_accuracy= []
    bootstrapped_f1_score= []
    bootstrapped_f2_score= [] 
    bootstrapped_roc_auc= []   
    
    for i in range(n_bootstraps): 
      index= rng_seed.randint(0, y_tests.shape[0], y_tests.shape[0])  
      if len(np.unique(y_tests[index])) < 2:  
        continue  
    
      tps=[]
      fps=[]
      fns=[]
      tns=[]

      for i in range (index.shape[0]):
          pred= 0 if preds[index][i][0] < threshold else 1
          if pred == 1 and y_tests[index][i] == 1:
            tps.append(preds[index][i][0])
          if pred == 1 and y_tests[index][i] == 0:
            fps.append(preds[index][i][0])
          if pred== 0 and y_tests[index][i] == 1:
            fns.append(preds[index][i][0])
          if pred == 0 and y_tests[index][i] == 0:
            tns.append(preds[index][i][0])
  
      tp= len(tps)
      fp= len(fps)
      fn= len(fns)
      tn= len(tns)

      sensitivity= tp/(tp+fn)
      specificity= tn/(tn+fp)
      precission= tp/(tp+fp)
      NPV=tn/(tn+fn)
      accuracy= (tp+tn)/(tp+fn+fp+tn)
      f1_score= 2*sensitivity*precission/(sensitivity+precission)
      f2_score= 5*sensitivity*precission/(sensitivity+4*precission)

      bootstrapped_sensitivity.append(sensitivity)
      bootstrapped_specificity.append(specificity)
      bootstrapped_precission.append(precission)
      bootstrapped_NPV.append(NPV)
      bootstrapped_accuracy.append(accuracy)
      bootstrapped_f1_score.append(f1_score)
      bootstrapped_f2_score.append(f2_score)


      roc_auc= sklearn.metrics.roc_auc_score(y_tests[index], preds[index])  
      bootstrapped_roc_auc.append(roc_auc)

    
    bootstrapped_sensitivity= np.array(bootstrapped_sensitivity)
    bootstrapped_sensitivity.sort()
    bootstrapped_specificity= np.array(bootstrapped_specificity)
    bootstrapped_specificity.sort()
    bootstrapped_precission= np.array(bootstrapped_precission)
    bootstrapped_precission.sort()
    bootstrapped_NPV= np.array(bootstrapped_NPV)
    bootstrapped_NPV.sort()
    bootstrapped_accuracy= np.array(bootstrapped_accuracy)
    bootstrapped_accuracy.sort()
    bootstrapped_f1_score= np.array(bootstrapped_f1_score)
    bootstrapped_f1_score.sort()
    bootstrapped_f2_score= np.array(bootstrapped_f2_score)
    bootstrapped_f2_score.sort()
    

    sorted_roc_auc_scores= np.array(bootstrapped_roc_auc) 
    sorted_roc_auc_scores.sort() 
    

    b_sensitivity= np.average(bootstrapped_sensitivity)
    lower_sensitivity= bootstrapped_sensitivity[int(((1-confidence)/2) * len(bootstrapped_sensitivity))]
    upper_sensitivity= bootstrapped_sensitivity[int((confidence + ((1-confidence)/2)) * len(bootstrapped_sensitivity))]
    b_specificity= np.average(bootstrapped_specificity)
    lower_specificity= bootstrapped_specificity[int(((1-confidence)/2) * len(bootstrapped_specificity))]
    upper_specificity= bootstrapped_specificity[int((confidence + ((1-confidence)/2)) * len(bootstrapped_specificity))]
    b_precission= np.average(bootstrapped_precission)
    lower_precission= bootstrapped_precission[int(((1-confidence)/2) * len(bootstrapped_precission))]
    upper_precission= bootstrapped_precission[int((confidence + ((1-confidence)/2)) * len(bootstrapped_precission))]
    b_NPV= np.average(bootstrapped_NPV)
    lower_NPV= bootstrapped_NPV[int(((1-confidence)/2) * len(bootstrapped_NPV))]
    upper_NPV= bootstrapped_NPV[int((confidence + ((1-confidence)/2)) * len(bootstrapped_NPV))]
    b_accuracy= np.average(bootstrapped_accuracy)
    lower_accuracy= bootstrapped_accuracy[int(((1-confidence)/2) * len(bootstrapped_accuracy))]
    upper_accuracy= bootstrapped_accuracy[int((confidence + ((1-confidence)/2)) * len(bootstrapped_accuracy))]
    b_f1_score= np.average(bootstrapped_f1_score)
    lower_f1_score= bootstrapped_f1_score[int(((1-confidence)/2) * len(bootstrapped_f1_score))]
    upper_f1_score= bootstrapped_f1_score[int((confidence + ((1-confidence)/2)) * len(bootstrapped_f1_score))]
    b_f2_score= np.average(bootstrapped_f2_score)
    lower_f2_score= bootstrapped_f2_score[int(((1-confidence)/2) * len(bootstrapped_f2_score))]
    upper_f2_score= bootstrapped_f2_score[int((confidence + ((1-confidence)/2)) * len(bootstrapped_f2_score))]  


    roc_auc_bootstraping= np.average(np.array(bootstrapped_roc_auc))
    lower_roc_auc_difference= sorted_roc_auc_scores[int(((1-confidence)/2) * len(sorted_roc_auc_scores))]
    upper_roc_auc_difference= sorted_roc_auc_scores[int((confidence + ((1-confidence)/2)) * len(sorted_roc_auc_scores))]    
    
   
    print('Sensitivity=%.3f, CI: lower=%.3f, upper=%.3f'%(b_sensitivity, lower_sensitivity, upper_sensitivity))
    print('Specificity=%.3f, CI: lower=%.3f, upper=%.3f'%(b_specificity, lower_specificity, upper_specificity))
    print('Precission=%.3f, CI: lower=%.3f, upper=%.3f'%(b_precission, lower_precission, upper_precission))
    print('NPV=%.3f, CI: lower=%.3f, upper=%.3f'%(b_NPV, lower_NPV, upper_NPV))
    print('Acurracy=%.3f, CI: lower=%.3f, upper=%.3f'%(b_accuracy, lower_accuracy, upper_accuracy))
    print('F1_score=%.3f, CI: lower=%.3f, upper=%.3f'%(b_f1_score, lower_f1_score, upper_f1_score))
    print('F2_score=%.3f, CI: lower=%.3f, upper=%.3f'%(b_f2_score, lower_f2_score, upper_f2_score))

    print('ROC_AUC=%.3f, CI: lower=%.3f, upper=%.3f'%(roc_auc_bootstraping, lower_roc_auc_difference, upper_roc_auc_difference))

In [ ]:
b_metrics= bootstrapping_metrics (predictions, labels_test_array, 0.95, 1000, 0.9)

Sensitivity=0.942, CI: lower=0.903, upper=0.975
Specificity=0.862, CI: lower=0.784, upper=0.933
Precission=0.915, CI: lower=0.866, upper=0.957
NPV=0.904, CI: lower=0.840, upper=0.960
Acurracy=0.911, CI: lower=0.875, upper=0.945
F1_score=0.928, CI: lower=0.897, upper=0.957
F2_score=0.936, CI: lower=0.903, upper=0.966
ROC_AUC=0.941, CI: lower=0.904, upper=0.971
